In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [19]:

class RandomDataset(Dataset):

    def __init__(self, input_size, data_size):
        self.len = data_size
        self.data = torch.randn(data_size, input_size)
        self.target = torch.randint(0, 1, (data_size,), dtype=torch.long)

    def __getitem__(self, index):
        return {"data":self.data[index],
                "target":self.target[index]
        }

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size,bias=True)

    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output
    


In [20]:
model = Model(input_size, output_size)
print(model)

if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model)

model.to(device)


Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)


Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

In [ ]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()

for batch in rand_loader:
      print(batch["data"].shape)
      print(batch["target"].shape)
      input = batch["data"].to(device)
      output = model(input)
      output_loss = loss(output, batch["target"].to(device))
      print(output_loss.item())
      
      print("Outside: input size", input.size(),
            "output_size", output.size())
      
    

torch.Size([30, 5])
torch.Size([30])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
0.7822713255882263
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
